In [159]:
from os import path
from itertools import groupby
from functools import reduce
import parse

instructions = list()

with open(path.join(globals()['_dh'][0], "input.txt")) as f:
    instructions = [x.strip() for x in f.readlines()]

instructions_test = list()
with open(path.join(globals()['_dh'][0], "test.txt")) as f:
    instructions_test = [x.strip() for x in f.readlines()]

monkey_index_pattern = parse.compile('Monkey {index}:')
starting_items_pattern = parse.compile('Starting items: {items}')
operation_pattern = parse.compile('Operation: new = {item1} {operand} {item2}')
test_pattern = parse.compile('Test: divisible by {modulo_base}')
condition_pattern = parse.compile('If {result}: throw to monkey {nextMonkey}')

In [107]:
def parse_monkey_index(monkey_index_raw):
    parsed = monkey_index_pattern.parse(monkey_index_raw)

    return int(parsed['index'])

In [38]:
def parse_starting_items(starting_items_raw):
    parsed = starting_items_pattern.parse(starting_items_raw)

    return [int(i) for i in parsed['items'].split(', ')]

In [64]:
def parse_operation(operation_raw):
    parsed = operation_pattern.parse(operation_raw)

    item_1 = parsed['item1']
    operand = parsed['operand']
    item_2 = parsed['item2']

    if item_1 == 'old' and item_2 == 'old':
        if operand == '+':
            return lambda x: x + x
        if operand == '-':
            return lambda x: 0
        if operand == '*':
            return lambda x: x ** 2
    if item_1 == 'old' and item_2 != 'old':
        if operand == '+':
            return lambda x: x + int(item_2)
        if operand == '-':
            return lambda x: x - int(item_2)
        if operand == '*':
            return lambda x: x * int(item_2)
    if item_1 != 'old' and item_2 == 'old':
        if operand == '+':
            return lambda x: int(item_2) + x
        if operand == '-':
            return lambda x: int(item_2) - x
        if operand == '*':
            return lambda x: int(item_2) * x
    if item_1 != 'old' and item_2 != 'old':
        if operand == '+':
            return lambda x: int(item_1) + int(item_2)
        if operand == '-':
            return lambda x: int(item_1) - int(item_2)
        if operand == '*':
            return lambda x: int(item_1) * int(item_2)

In [97]:
def parse_test(test_raw, truthy_condition_raw, falsy_condition_raw):
    parsed = test_pattern.parse(test_raw)

    modulo_base = int(parsed['modulo_base'])

    truthy_monkey_index = int(condition_pattern.parse(truthy_condition_raw)['nextMonkey'])
    falsy_monkey_index = int(condition_pattern.parse(falsy_condition_raw)['nextMonkey'])

    return lambda x: truthy_monkey_index if x % modulo_base == 0 else falsy_monkey_index

In [156]:
def parse_input(input_list):
    monkeys = dict()

    grouped = [list(g) for k, g in groupby(input_list, lambda s: s == '') if not k]

    for group in grouped:
        monkey_index_raw, starting_items_raw, operation_raw, test_raw, truthy_condition_raw, falsy_condition_raw = tuple(
            group)

        monkey_index = parse_monkey_index(monkey_index_raw)
        starting_items = parse_starting_items(starting_items_raw)
        operation = parse_operation(operation_raw)
        test_condition = parse_test(test_raw, truthy_condition_raw, falsy_condition_raw)

        monkeys[monkey_index] = {
            'items': starting_items,
            'operation': operation,
            'test_condition': test_condition,
            'inspection_count': 0
        }

    return monkeys

In [157]:
def part1(input_list):
    monkeys = parse_input(input_list)

    monkey_count = max(monkeys.keys()) + 1

    rounds = 20

    for _ in range(rounds):
        for monkeyIndex in range(monkey_count):
            items = monkeys[monkeyIndex]['items']
            monkeys[monkeyIndex]['inspection_count'] += len(items)

            for itemIndex in range(len(items)):
                item = items.pop(0)
                new_worry_level = monkeys[monkeyIndex]['operation'](item)

                rounded = new_worry_level // 3

                next_monkey = monkeys[monkeyIndex]['test_condition'](rounded)

                monkeys[next_monkey]['items'].append(rounded)
    most_inspections = sorted([v['inspection_count'] for v in monkeys.values()])[-2:]

    return reduce(lambda a, b: a * b, most_inspections, 1)

In [176]:
def part2(input_list):
    monkeys = parse_input(input_list)

    monkey_count = max(monkeys.keys()) + 1

    rounds = 95

    for _ in range(rounds):
        for monkeyIndex in range(monkey_count):
            items = monkeys[monkeyIndex]['items']
            monkeys[monkeyIndex]['inspection_count'] += len(items)

            for itemIndex in range(len(items)):
                item = items.pop(0)
                new_worry_level = monkeys[monkeyIndex]['operation'](item)

                next_monkey = monkeys[monkeyIndex]['test_condition'](new_worry_level)

                monkeys[next_monkey]['items'].append(new_worry_level)
    most_inspections = sorted([v['inspection_count'] for v in monkeys.values()])[-2:]

    return reduce(lambda a, b: a * b, most_inspections, 1)

In [177]:
print(f"Solution to part 1 is : {part1(instructions)}")
print(f"Solution to part 2 is : {part2(instructions_test)}")

Solution to part 1 is : 316888
Solution to part 2 is : 235224
